In [1]:
## Imports

import sys
import os
import time
import math
import random
import pdb
import h5py

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if par_dir not in sys.path:
    sys.path.append(par_dir)
    
# Plotting import
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Import the utils for plotting the metrics
from plot_utils import plot_utils
from plot_utils import notebook_utils_2

from sklearn.metrics import roc_curve, auc

In [2]:
# Dictionary mapping the ordinal labels to particle types 
LABEL_DICT = {0:"gamma", 1:"e", 2:"mu"}

# Fix the colour scheme for each particle type
COLOR_DICT = {"gamma":"red", "e":"blue", "mu":"green"}

In [3]:
# Get ResNet-18 results

fprs = []
tprs = []
thresholds = []

run_ids = ["20200418_095532"]

dump_dir = "/home/ttuinstr/VAE/dumps/"
dump_file = "/test_validation_iteration_dump.npz"

softmax_index_dict = {value:key for key, value in LABEL_DICT.items()}

for run_id in run_ids:
    
    test_dump_path = dump_dir + run_id + dump_file
    test_dump_np = np.load(test_dump_path, allow_pickle=True)
    
    res_predictedlabels = test_dump_np['predicted_labels'].flatten()
    res_softmaxes  = test_dump_np['softmax'].reshape(-1)
    res_labels   = test_dump_np['labels'].reshape(-1)
    res_energies = test_dump_np['energies'].reshape(-1)
    res_rootfiles = test_dump_np['rootfiles'].reshape(-1)
    res_eventids = test_dump_np['eventids'].reshape(-1)
    #res_positions = test_dump_np['positions'].reshape(-1)
    res_angles = test_dump_np['angles'].reshape(-1)

FileNotFoundError: [Errno 2] No such file or directory: '/home/ttuinstr/VAE/dumps/20200418_095532/test_validation_iteration_dump.npz'

In [8]:
#res_softmaxes2 = res_softmaxes[0]
#res_rootfiles2 = res_rootfiles[0]
#res_energies2 = res_energies[0]
res_predictedlabels2 = res_predictedlabels[0]


for i in np.arange(res_predictedlabels.shape[0] - 1):
    res_predictedlabels2 = np.concatenate((res_predictedlabels2, res_predictedlabels[i + 1]), axis=0)


#np.savez('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/resnet_softmaxes', res_softmaxes2)
#np.savez('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/resnet_rootfiles', res_rootfiles2)
#np.savez('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/resnet_eventids', res_eventids2)
#np.savez('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/resnet_energies', res_energies2)
#np.savez('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/resnet_predictedlabels', res_predictedlabels2)

# Mapping ResNet-18 results to original h5 files
'''
mapping_indices = np.array([])

for i in np.arange(filtered_rootfiles.shape[0]):
    id_index = np.where(res_eventids2 == filtered_eventids[i])[0]
    mapping_indices = np.append(mapping_indices, id_index[np.where((res_rootfiles2[id_index] == filtered_rootfiles[i]))[0]])

np.savez('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/map_indices_resnet', mapping_indices)      
    
print(mapping_indices.shape)
'''

"\nmapping_indices = np.array([])\n\nfor i in np.arange(filtered_rootfiles.shape[0]):\n    id_index = np.where(res_eventids2 == filtered_eventids[i])[0]\n    mapping_indices = np.append(mapping_indices, id_index[np.where((res_rootfiles2[id_index] == filtered_rootfiles[i]))[0]])\n\nnp.savez('/home/ttuinstr/VAE/debugging/map_indices_resnet', mapping_indices)      \n    \nprint(mapping_indices.shape)\n"

In [19]:
# Get original h5 file info

# Import test events from h5 file
filtered_index = "/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz"
filtered_indices = np.load(filtered_index, allow_pickle=True)
test_filtered_indices = filtered_indices['test_idxs']
print(test_filtered_indices.shape)

original_data_path = "/data/WatChMaL/data/IWCDmPMT_4pi_fulltank_9M.h5"
f = h5py.File(original_data_path, "r")

hdf5_event_data = (f["event_data"])
original_eventdata = np.memmap(original_data_path, mode="r", shape=hdf5_event_data.shape,
                                    offset=hdf5_event_data.id.get_offset(), dtype=hdf5_event_data.dtype)

original_eventids = np.array(f['event_ids'])
original_rootfiles = np.array(f['root_files'])
original_energies = np.array(f['energies'])
original_positions = np.array(f['positions'])
original_angles = np.array(f['angles'])
original_labels = np.array(f['labels'])

#filtered_eventdata = original_eventdata[test_filtered_indices]
filtered_eventids = original_eventids[test_filtered_indices]
filtered_rootfiles = original_rootfiles[test_filtered_indices]
filtered_energies = original_energies[test_filtered_indices]
filtered_positions = original_positions[test_filtered_indices]
filtered_angles = original_angles[test_filtered_indices]
filtered_labels = original_labels[test_filtered_indices]

(3351020,)


In [20]:
# Get FiTQun results

# File paths for fiTQun results
fiTQun_e_path = "/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_fiTQun_e-.npz"
fiTQun_mu_path = "/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_fiTQun_mu-.npz"
fiTQun_gamma_path = "/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_fulltank_fiTQun_gamma.npz"

# Load fiTQun results
f_e = np.load(fiTQun_e_path, allow_pickle=True)
f_mu = np.load(fiTQun_mu_path, allow_pickle=True)
f_gamma = np.load(fiTQun_gamma_path, allow_pickle=True)
#list(f_e.keys())

# Load the results

# Remove events with a non-zero flag (this filtering will be applied to the other results as well)
# * A non-zero flag value usually implies that either the reconstruction is known to have failed 
#   or the particle exited the tank and so would not be included in actual physics analysis
e_flag = np.array(f_e['flag'])
e_indices = np.where((e_flag[:,0] == 0) & (e_flag[:,1] == 0))
mu_flag = np.array(f_mu['flag'])
mu_indices = np.where((mu_flag[:,0] == 0) & (mu_flag[:,1] == 0))
gamma_flag = np.array(f_gamma['flag'])
gamma_indices = np.where((gamma_flag[:,0] == 0) & (gamma_flag[:,1] == 0))

# Get the filename for each event
e_file = np.array(f_e['filename'])
e_file = e_file[e_indices]
mu_file = np.array(f_mu['filename'])
mu_file = mu_file[mu_indices]
gamma_file = np.array(f_gamma['filename'])
gamma_file = gamma_file[gamma_indices]

# Get the event ID for each event
e_eventid = np.array(f_e['eventid'])
e_eventid = e_eventid[e_indices]
mu_eventid = np.array(f_mu['eventid'])
mu_eventid = mu_eventid[mu_indices]
gamma_eventid = np.array(f_gamma['eventid'])
gamma_eventid = gamma_eventid[gamma_indices]

In [21]:
# Map ResNet results to fiTQun events

# Map ResNet results to original h5 file events
mapping_indices = np.load('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/map_indices_resnet.npz')
mapping_indices = mapping_indices['arr_0'].astype(int)

res_softmaxes = np.load('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/resnet_softmaxes.npz')
res_softmaxes = res_softmaxes['arr_0']
res_energies = np.load('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/resnet_energies.npz')
res_energies = res_energies['arr_0']

res_softmaxes = res_softmaxes[mapping_indices]
res_energies = res_energies[mapping_indices]

# Separate event types
e_test_indices = np.load('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/test_indices_e.npz')
e_test_indices = e_test_indices['arr_0'].astype(int)
mu_test_indices = np.load('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/test_indices_mu.npz')
mu_test_indices = mu_test_indices['arr_0'].astype(int)
gamma_test_indices = np.load('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/test_indices_gamma.npz')
gamma_test_indices = gamma_test_indices['arr_0'].astype(int)

e_softmaxes = res_softmaxes[e_test_indices]
mu_softmaxes = res_softmaxes[mu_test_indices]
gamma_softmaxes = res_softmaxes[gamma_test_indices]

e_labels = filtered_labels[e_test_indices]
mu_labels = filtered_labels[mu_test_indices]
gamma_labels = filtered_labels[gamma_test_indices]

e_rootfiles = filtered_rootfiles[e_test_indices]
mu_rootfiles = filtered_rootfiles[mu_test_indices]
gamma_rootfiles = filtered_rootfiles[gamma_test_indices]

e_eventids = filtered_eventids[e_test_indices]
mu_eventids = filtered_eventids[mu_test_indices]
gamma_eventids = filtered_eventids[gamma_test_indices]

e_positions = filtered_positions[e_test_indices]
mu_positions = filtered_positions[mu_test_indices]
gamma_positions = filtered_positions[gamma_test_indices]

e_angles = filtered_angles[e_test_indices]
mu_angles = filtered_angles[mu_test_indices]
gamma_angles = filtered_angles[gamma_test_indices]

e_energies = filtered_energies[e_test_indices]
mu_energies = filtered_energies[mu_test_indices]
gamma_energies = filtered_energies[gamma_test_indices]

# Match events in event types to fiTQun results
e_map_indices = np.load('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/map_indices_e.npz')
e_map_indices = e_map_indices['arr_0'].astype(int)
mu_map_indices = np.load('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/map_indices_mu.npz')
mu_map_indices = mu_map_indices['arr_0'].astype(int)
gamma_map_indices = np.load('/home/ttuinstr/VAE/Comparing_ResNet_and_FiTQun/map_indices_gamma.npz')
gamma_map_indices = gamma_map_indices['arr_0'].astype(int)


#e_eventdata = np.sum(original_eventdata[test_filtered_indices][e_test_indices][e_map_indices] > 0, (1,2,3))
#mu_eventdata = np.sum(original_eventdata[test_filtered_indices][mu_test_indices][mu_map_indices] > 0, (1,2,3))
#gamma_eventdata = np.sum(original_eventdata[test_filtered_indices][gamma_test_indices][gamma_map_indices] > 0, (1,2,3))

e_softmaxes = e_softmaxes[e_map_indices]
mu_softmaxes = mu_softmaxes[mu_map_indices]
gamma_softmaxes = gamma_softmaxes[gamma_map_indices]

e_positions = e_positions[e_map_indices]
mu_positions = mu_positions[mu_map_indices]
gamma_positions = gamma_positions[gamma_map_indices]

e_angles = e_angles[e_map_indices]
mu_angles = mu_angles[mu_map_indices]
gamma_angles = gamma_angles[gamma_map_indices]

e_energies = e_energies[e_map_indices]
mu_energies = mu_energies[mu_map_indices]
gamma_energies = gamma_energies[gamma_map_indices]